<a href="https://colab.research.google.com/github/ravi-gopalan/ds_bowl/blob/master/2019_ds_bowl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
cd /root

/root


In [0]:
!mkdir .kaggle

In [0]:
!cp '/gdrive/My Drive/Colab_stuff/kaggle.json' '/root/.kaggle/kaggle.json' 
!chmod 600 /root/.kaggle/kaggle.json

In [5]:
cd /content/

/content


In [0]:
!mkdir data_science_bowl_2019

In [7]:
cd data_science_bowl_2019

/content/data_science_bowl_2019


In [8]:
!kaggle competitions download -c data-science-bowl-2019

 81% 33.0M/40.8M [00:00<00:00, 49.6MB/s]
100% 40.8M/40.8M [00:00<00:00, 103MB/s] 
 96% 380M/397M [00:04<00:00, 72.1MB/s]
100% 397M/397M [00:05<00:00, 81.6MB/s]
  0% 0.00/399k [00:00<?, ?B/s]
100% 399k/399k [00:00<00:00, 134MB/s]
  0% 0.00/262k [00:00<?, ?B/s]
100% 262k/262k [00:00<00:00, 85.0MB/s]
  0% 0.00/10.8k [00:00<?, ?B/s]
100% 10.8k/10.8k [00:00<00:00, 9.44MB/s]


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import gc
import json
import timeit

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'


In [0]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [11]:
cd /content/data_science_bowl_2019

/content/data_science_bowl_2019


In [12]:
!unzip train.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               


In [13]:
# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) \
        for x in dir() if not x.startswith('_') \
        and x not in sys.modules \
        and x not in ipython_vars], \
       key=lambda x: x[1], reverse=True)

[('InteractiveShell', 1056),
 ('reduce_mem_usage', 136),
 ('drive', 80),
 ('np', 80),
 ('pd', 80),
 ('plt', 80),
 ('sns', 80)]

In [14]:
df_train = pd.read_csv('train.csv',usecols=['event_data'])
df_train.shape

(11341042, 1)

In [0]:
chunk_size = 1000000

In [0]:
def create_csv_file(df, chunksize, start, end,output_csv):
  event = pd.io.json.json_normalize(df[chunksize*start:chunksize*end].event_data.apply(json.loads),max_level=1)
  event.shape
  event.head()
  event.tail()
  event.to_csv(output_csv)
  del event
  gc.collect()
  return



In [73]:
start_time = timeit.default_timer()
create_csv_file(df_train,chunk_size,10,11,'event_11.csv')
stop_time = timeit.default_timer()

stop_time - start_time

121.50837158700051

In [74]:
event_12 = pd.io.json.json_normalize(df_train[11*chunk_size:].event_data.apply(json.loads),max_level=1)
event_12.shape
event_12.head()
event_12.tail()
event_12.to_csv('event_12.csv')
del event_12
gc.collect()

(341042, 134)

,event_count,game_time,event_code,coordinates.x,coordinates.y,coordinates.stage_width,coordinates.stage_height,description,identifier,media_type,total_duration,duration,cloud,cloud_size,water_level,filled,version,round,tutorial_step,shells,holes,holding_shell,has_water,shell_size,position,movie_id,target_containers,resources,container_type,containers,correct,current_containers,total_containers,misses,toy_earned,object,previous_jars,bottles,source,bottle.amount,...,object_type,group,bug,buglength,stage_number,hat,caterpillar,hats,caterpillars,bird_height,hats_placed,target_weight,table_weights,starting_weights,scale_weights,location,weights,pillars,diet,scale_weight,scale_contents,houses,dinosaurs,dinosaurs_placed,house.size,house.position,dinosaur_weight,dinosaur_count,molds,total_bowls,bowls,has_toy,bowl_id,toy,max_position,end_position,tape_length,bug_length,dwell_time,chests
0,53,86825.0,4070,192.0,416.0,1015.0,762.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,54,86991.0,4070,796.0,392.0,1015.0,762.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,55,91034.0,3010,NaN,NaN,NaN,NaN,Keep making it rain!,Buddy_KeepMakingItRain,audio,1610.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,56,92809.0,3110,NaN,NaN,NaN,NaN,Keep making it rain!,Buddy_KeepMakingItRain,audio,NaN,1775.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,57,93821.0,4020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,event_count,game_time,event_code,coordinates.x,coordinates.y,coordinates.stage_width,coordinates.stage_height,description,identifier,media_type,total_duration,duration,cloud,cloud_size,water_level,filled,version,round,tutorial_step,shells,holes,holding_shell,has_water,shell_size,position,movie_id,target_containers,resources,container_type,containers,correct,current_containers,total_containers,misses,toy_earned,object,previous_jars,bottles,source,bottle.amount,...,object_type,group,bug,buglength,stage_number,hat,caterpillar,hats,caterpillars,bird_height,hats_placed,target_weight,table_weights,starting_weights,scale_weights,location,weights,pillars,diet,scale_weight,scale_contents,houses,dinosaurs,dinosaurs_placed,house.size,house.position,dinosaur_weight,dinosaur_count,molds,total_bowls,bowls,has_toy,bowl_id,toy,max_position,end_position,tape_length,bug_length,dwell_time,chests
341037,66,118615.0,3110,NaN,NaN,NaN,NaN,Tap another dinosaur.,clip_307_touch,animation,NaN,1450.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
341038,1,NaN,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
341039,1,NaN,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
341040,1,NaN,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
341041,1,NaN,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


11

In [0]:
from google.colab import files


In [76]:
del df_train
gc.collect()

294

28

In [77]:
# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) \
        for x in dir() if not x.startswith('_') \
        and x not in sys.modules \
        and x not in ipython_vars], \
       key=lambda x: x[1], reverse=True)

[('InteractiveShell', 1056),
 ('create_csv_file', 136),
 ('reduce_mem_usage', 136),
 ('drive', 80),
 ('np', 80),
 ('pd', 80),
 ('plt', 80),
 ('sns', 80),
 ('chunk_size', 28),
 ('start_time', 24),
 ('stop_time', 24)]

In [15]:
train = pd.read_csv('train.csv',usecols=['event_data'], iterator=True)
type(train)

pandas.io.parsers.TextFileReader

In [0]:
df_empty = pd.DataFrame()
for chunk in train:
  df_empty = pd.concat([df_empty, pd.io.json.json_normalize(chunk.event_data.apply(json.loads), max_level=1)])

In [27]:
train.get_chunk(5)

,event_data
52,"{""coordinates"":{""x"":157,""y"":188,""stage_width"":..."
53,"{""coordinates"":{""x"":196,""y"":168,""stage_width"":..."
54,"{""coordinates"":{""x"":588,""y"":379,""stage_width"":..."
55,"{""coordinates"":{""x"":800,""y"":243,""stage_width"":..."
56,"{""size"":0,""coordinates"":{""x"":782,""y"":207,""stag..."


In [30]:

n_chunks = 0
for chunk in train:
  n_chunks += 1
print(n_chunks)

23


In [32]:
df_list = ['df_'+str(i) for i in range(23)]
df_list

['df_0',
 'df_1',
 'df_2',
 'df_3',
 'df_4',
 'df_5',
 'df_6',
 'df_7',
 'df_8',
 'df_9',
 'df_10',
 'df_11',
 'df_12',
 'df_13',
 'df_14',
 'df_15',
 'df_16',
 'df_17',
 'df_18',
 'df_19',
 'df_20',
 'df_21',
 'df_22']

In [0]:
for df, chunk in zip(df_list, train):
   df = pd.io.json.json_normalize(chunk.event_data.apply(json.loads))


In [35]:
type(df_list[0])

str

In [15]:
train.get_chunk(1)

StopIteration: ignored

In [0]:
extracted_events = pd.io.json.json_normalize(df_train.event_data.apply(json.loads))